In [4]:
%pip install pillow==10.3.0
%pip install numpy==1.26.4

   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 2.5/2.5 MB 16.1 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: pillow 11.2.1
    Uninstalling pillow-11.2.1:
      Successfully uninstalled pillow-11.2.1
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:

!pip install toxigen datasets -q
!pip install evaluate -q

  Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl (15.5 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6
Note: you may need to restart the kernel to use updated packages.


  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.37.1 requires pillow<11,>=7.1.0, but you have pillow 11.2.1 which is incompatible.


Note: you may need to restart the kernel to use updated packages.
  Using cached numpy-2.2.6-cp312-cp312-win_amd64.whl.metadata (60 kB)
Using cached numpy-2.2.6-cp312-cp312-win_amd64.whl (12.6 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
contourpy 1.2.0 requires numpy<2.0,>=1.20, but you have numpy 2.2.6 which is incompatible.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.2.6 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.6 which is incompatible.
streamlit 1.37.1 requires pillow<11,>=7.1.0, but you have pillow 11.2.1 which is incompatible.


In [7]:
# If you see "ImportError: numpy.core.multiarray failed to import", 
# restart the kernel after running cell 0 to reload numpy.
# Cell 0 reinstalls numpy, so a restart is required before running this cell.
# !!! IMPORTANT: After running cell 0, restart the Jupyter kernel before running this cell !!!

# !!! If you have not restarted the kernel after running cell 0, please do so now !!!
# Otherwise, you may encounter ImportError related to numpy.

from loading_dataset_toxigen import load_toxigen, post_process_toxigen
from hatebert_model_for_docker import load_hatebert, tokenize_function, train_epoch_hatebert, evaluate_hatebert, evaluate_hatebert_with_bias
from sc_weight_BERT_model import load_sc_weight_BERT
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import pandas as pd
from sklearn.metrics import f1_score, accuracy_score
import argparse

import os
from pathlib import Path


ImportError: cannot import name 'load_sc_weight_BERT' from 'sc_weight_BERT_model' (c:\Documents\EPFL\Cours\MA2\Deep learning\project\deep_learning2025\sc_weight_BERT_model.py)

In [1]:
from hateXplain_dataset import HateXplainDataset

In [8]:
toxigen = load_toxigen()

In [9]:
print(toxigen)

DatasetDict({
    test: Dataset({
        features: ['text', 'target_group', 'factual?', 'ingroup_effect', 'lewd', 'framing', 'predicted_group', 'stereotyping', 'intent', 'toxicity_ai', 'toxicity_human', 'predicted_author', 'actual_method', 'labels'],
        num_rows: 940
    })
    train: Dataset({
        features: ['text', 'target_group', 'factual?', 'ingroup_effect', 'lewd', 'framing', 'predicted_group', 'stereotyping', 'intent', 'toxicity_ai', 'toxicity_human', 'predicted_author', 'actual_method', 'labels'],
        num_rows: 8960
    })
})


In [2]:
# Create an instance of the class
hatexplain = HateXplainDataset()

In [3]:
# Load the dataset (this will also process the labels)
dataset = hatexplain.load_dataset()

# Print the dataset (this prints the HuggingFace DatasetDict object)
print(dataset)

INFO:hateXplain_dataset:Attempting to load HateXplain dataset...
INFO:hateXplain_dataset:Processed 15383 examples from train split
INFO:hateXplain_dataset:Processed 1922 examples from validation split
INFO:hateXplain_dataset:Processed 1924 examples from test split
INFO:hateXplain_dataset:Dataset loaded successfully
INFO:hateXplain_dataset:Available splits: ['train', 'validation', 'test']


DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'target_group'],
        num_rows: 15383
    })
    validation: Dataset({
        features: ['text', 'labels', 'target_group'],
        num_rows: 1922
    })
    test: Dataset({
        features: ['text', 'labels', 'target_group'],
        num_rows: 1924
    })
})


In [4]:
# If you want to print statistics (using your class's method):
# Note: The print_dataset_statistics method expects self._processed_dataset to be set.
# You can set it manually after loading:
hatexplain._processed_dataset = dataset
hatexplain.print_dataset_statistics()


Dataset Statistics:

TRAIN Split Statistics:
------------------------------
Total examples: 15383

Label Distribution:
Hate Speech: 6251 (40.64%)
Normal/Offensive: 9132 (59.36%)

Target Group Distribution:
None: 5509 (35.81%)
African: 2335 (15.18%)
Homosexual: 1161 (7.55%)
Islam: 1156 (7.51%)
Jewish: 1011 (6.57%)
Women: 898 (5.84%)
Other: 883 (5.74%)
Arab: 603 (3.92%)
Refugee: 529 (3.44%)
Caucasian: 386 (2.51%)
Asian: 269 (1.75%)
Men: 236 (1.53%)
Hispanic: 187 (1.22%)
Disability: 53 (0.34%)
Christian: 52 (0.34%)
Heterosexual: 31 (0.20%)
Minority: 24 (0.16%)
Economic: 23 (0.15%)
Indian: 15 (0.10%)
Hindu: 11 (0.07%)
none: 7 (0.05%)
Indigenous: 2 (0.01%)
Bisexual: 1 (0.01%)
Buddhism: 1 (0.01%)

Text Length Statistics:
Average length: 23.47 words
Maximum length: 165 words
Minimum length: 2 words

Hate Speech Distribution by Target Group:
None: 4410 hate speech examples (80.05% of this target)
Other: 253 hate speech examples (28.65% of this target)
Homosexual: 238 hate speech examples (20.

In [ ]:
#load sc_weight_BERT

In [6]:
train_HateXplain_dataset = dataset["train"]
eval_HateXplain_dataset = dataset["test"]

In [7]:
print(train_HateXplain_dataset)

Dataset({
    features: ['text', 'labels', 'target_group'],
    num_rows: 15383
})


In [6]:
tokenzied_HateXplain = hatexplain.post_process_HateXplain(dataset)

TypeError: HateXplainDataset.post_process_HateXplain() takes 1 positional argument but 2 were given

In [10]:
tokenizer, model_hatebert = load_hatebert()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of the model checkpoint at GroNLP/hateBERT were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some we

In [11]:
'''
 apply the function to all the elements in the dataset (individually or in batches)
 https://huggingface.co/docs/datasets/v1.11.0/package_reference/main_classes.html?highlight=dataset%20map#datasets.Dataset.map
 batch mode is very powerful. It allows you to speed up processing
 more info here: https://huggingface.co/docs/datasets/en/about_map_batch
'''
cache_files = {
    "test": ".cache/datasets/toxigen/toxigen_test_tokenized.arrow",
    "train": ".cache/datasets/toxigen/toxigen_train_tokenized.arrow"
} #path to the local cache files, where the current computation from the following function will be stored. 
# Caching saves RAM when working with large datasets and saves time instead of doing transformations on the fly.
tokenized_toxigen = toxigen.map(lambda x: tokenize_function(tokenizer, x, "text"), batched=True, cache_file_names=cache_files)

In [12]:
print(tokenized_toxigen)


DatasetDict({
    test: Dataset({
        features: ['text', 'target_group', 'factual?', 'ingroup_effect', 'lewd', 'framing', 'predicted_group', 'stereotyping', 'intent', 'toxicity_ai', 'toxicity_human', 'predicted_author', 'actual_method', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 940
    })
    train: Dataset({
        features: ['text', 'target_group', 'factual?', 'ingroup_effect', 'lewd', 'framing', 'predicted_group', 'stereotyping', 'intent', 'toxicity_ai', 'toxicity_human', 'predicted_author', 'actual_method', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 8960
    })
})


In [13]:
tokenized_toxigen = post_process_toxigen(tokenized_toxigen)

In [14]:
print(tokenized_toxigen)

DatasetDict({
    test: Dataset({
        features: ['target_group', 'factual?', 'ingroup_effect', 'lewd', 'framing', 'predicted_group', 'stereotyping', 'intent', 'toxicity_ai', 'toxicity_human', 'predicted_author', 'actual_method', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 940
    })
    train: Dataset({
        features: ['target_group', 'factual?', 'ingroup_effect', 'lewd', 'framing', 'predicted_group', 'stereotyping', 'intent', 'toxicity_ai', 'toxicity_human', 'predicted_author', 'actual_method', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 8960
    })
})


In [39]:
# create a smaller subset of the dataset as previously shown to speed up the fine-tuning

small_train_dataset = tokenized_toxigen["train"].shuffle(seed=42).select(range(50))
small_eval_dataset = tokenized_toxigen["test"].shuffle(seed=42).select(range(50))

train_dataset = tokenized_toxigen["train"]
eval_dataset = tokenized_toxigen["test"]

In [40]:
# create a DataLoader for your training and test datasets so you can iterate over batches of data:
train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=8)
test_dataloader = DataLoader(small_eval_dataset, batch_size=8)

In [41]:
print("HateBERT number of parameters: ", model_hatebert.num_parameters())

HateBERT number of parameters:  109483778


In [42]:
from torch.optim import AdamW

optimizer = AdamW(model_hatebert.parameters(), lr=5e-5)

In [43]:
from transformers import get_scheduler

num_epochs = 2
num_training_steps = num_epochs * len(train_dataloader)
# feel free to experiment with different num_warmup_steps
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=1, num_training_steps=num_training_steps
)

In [44]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model_hatebert.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [45]:
train_loss, train_metrics = train_epoch_hatebert(model_hatebert, num_epochs, train_dataloader, optimizer, lr_scheduler, device, num_training_steps)

  0%|          | 0/14 [00:00<?, ?it/s]

Batch type: <class 'dict'>


  7%|▋         | 1/14 [00:29<06:26, 29.76s/it]

Batch type: <class 'dict'>


 14%|█▍        | 2/14 [00:53<05:17, 26.43s/it]

Batch type: <class 'dict'>


 21%|██▏       | 3/14 [01:16<04:29, 24.52s/it]

Batch type: <class 'dict'>


KeyboardInterrupt: 

In [29]:
evaluate_hatebert(model_hatebert, test_dataloader, device)

  0%|          | 0/7 [00:00<?, ?it/s]

Batch type: <class 'dict'>


 14%|█▍        | 1/7 [00:05<00:35,  5.93s/it]

Batch type: <class 'dict'>


 29%|██▊       | 2/7 [00:11<00:29,  5.95s/it]

...

Batch type: <class 'dict'>


 86%|████████▌ | 6/7 [00:37<00:06,  6.19s/it]

Batch type: <class 'dict'>


100%|██████████| 7/7 [00:39<00:00,  5.61s/it]

{'accuracy': 0.64}


In [ ]:
overall_accuracy, epoch_loss, epoch_metrics = evaluate_hatebert_with_bias(model_hatebert, test_dataloader, device)

Batch type: <class 'dict'>
Batch type: <class 'dict'>
Batch type: <class 'dict'>
Batch type: <class 'dict'>
Batch type: <class 'dict'>
Batch type: <class 'dict'>
Batch type: <class 'dict'>
Overall Accuracy: 0.6400
Accuracy for target group 'asian folks': 1.0000
Number of samples in group 'asian folks': 1
Confusion Matrix for 'asian folks':
[[1]]

Accuracy for target group 'black folks / african-americans': 0.5000
Number of samples in group 'black folks / african-americans': 2
Confusion Matrix for 'black folks / african-americans':
[[1 0]
 [1 0]]

Accuracy for target group 'black/african-american folks': 0.2500
Number of samples in group 'black/african-american folks': 4
Confusion Matrix for 'black/african-american folks':
[[1 0]
 [3 0]]

Accuracy for target group 'chinese folks': 1.0000
Number of samples in group 'chinese folks': 3
Confusion Matrix for 'chinese folks':
[[3]]

Accuracy for target group 'folks with mental disabilities': 0.2500
Number of samples in group 'folks with menta

c:\Users\mikae\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:409: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
c:\Users\mikae\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:409: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
c:\Users\mikae\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:409: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
c:\Users\mikae\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:409: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct s

0.64

In [ ]:
'''
def train_cycle(model, optimizer, criterion, metrics, train_loader, test_loader, n_epochs, device,
                store_checkpoint_for_every_epoch, start_epoch=0):
    train_loss_log,  test_loss_log = [], []
    metrics_names = list(metrics.keys())
    train_metrics_log = [[] for i in range(len(metrics))]
    test_metrics_log = [[] for i in range(len(metrics))]


    for epoch in range(start_epoch, n_epochs):
        print("Epoch {0} of {1}".format(epoch, n_epochs - 1))
        train_loss, train_metrics = train_epoch_hatebert(model_hatebert, num_epochs, train_dataloader, optimizer, lr_scheduler, device, num_training_steps)(model, optimizer, criterion, metrics, train_loader, device)

        test_loss, test_metrics = evaluate_hatebert_with_bias(model, criterion, metrics, test_loader, device)

        train_loss_log.append(train_loss)
        train_metrics_log = update_metrics_log(metrics_names, train_metrics_log, train_metrics)

        test_loss_log.append(test_loss)
        test_metrics_log = update_metrics_log(metrics_names, test_metrics_log, test_metrics)

        plot_training(train_loss_log, test_loss_log, metrics_names, train_metrics_log, test_metrics_log)

        save_checkpoint(model, optimizer, epoch, loss=train_loss, checkpoint_path = Path(RESULTS_DIR) / "checkpoints/checkpoint.pth",
                        store_checkpoint_for_every_epoch=store_checkpoint_for_every_epoch)
    return train_metrics_log, test_metrics_log


saved_epoch, _ = load_checkpoint(model, optimizer, checkpoint_path = Path(RESULTS_DIR) / "checkpoints/checkpoint.pth")
if saved_epoch == 0:
    start_epoch = 0
else:
    start_epoch = saved_epoch + 1  #if the checkpoint from the epoch saved_epoch is stored, we want to start the training from the next epoch

criterion = nn.CrossEntropyLoss()

N_EPOCHS = 3
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device}")

model_hatebert.to(device)
criterion.to(device)


train_metrics_log, test_metrics_log = train_cycle(model_hatebert, optimizer, criterion, metrics, train_loader, valid_loader,
                                                  n_epochs=N_EPOCHS, device=device, store_checkpoint_for_every_epoch=False,
                                                  start_epoch=start_epoch)
'''
